In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cv2
import json
from tqdm import tqdm
import os

from src.pipeline.pipelinepredictor import PipelinePredictor
from src.pipeline.utils import get_image_visualization

import os
os.chdir('C:\\Users\\rusla\\Desktop\\ITMO\\7-sem\\thesis\\rhtr')

In [3]:
PIPELINE_CONFIG_PATH = 'src/pipeline/scripts/pipeline_config.json'

# Path to json with segmentation dataset annotation in COCO format
DATA_JSON_PATH = 'data/raw/school_notebooks_RU/annotations_test.json'

# Path to folder with evaluation images
IMAGE_ROOT = 'data/raw/school_notebooks_RU/images'

# Folder to save prediction json (one for each image)
SAVE_FOLDER = 'data/metrics'

In [4]:
predictor = PipelinePredictor(PIPELINE_CONFIG_PATH)

cwd='C:\\Users\\rusla\\Desktop\\ITMO\\7-sem\\thesis\\rhtr' config_path='models/segmentation/3/segm_config.json'


C:\Users\rusla\Desktop\ITMO\7-sem\thesis\rhtr\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\rusla\Desktop\ITMO\7-sem\thesis\rhtr\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
import numpy as np
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [14]:
os.makedirs(SAVE_FOLDER, exist_ok=True)

with open(DATA_JSON_PATH, 'r') as f:
    data = json.load(f)

for idx, data_img in enumerate(tqdm(data['images'])):
    if idx == 2: break
    img_name = data_img['file_name']
    image = cv2.imread(str(os.path.join(IMAGE_ROOT, img_name)))

    rotated_image, pred_data = predictor.predict(image)
    
    print(f"{pred_data['predictions'][0].keys()}")

    pred_json_name = os.path.splitext(img_name)[0] + '.json'
    pred_json_path = os.path.join(SAVE_FOLDER, pred_json_name)
    
    with open(pred_json_path, 'w') as f:
        json.dump(pred_data, f, cls=NumpyEncoder)

  0%|          | 0/150 [00:00<?, ?it/s]

dict_keys(['polygon', 'bbox', 'class_name', 'rotated_polygon', 'rotated_bbox', 'crop', 'text', 'polygon_center'])


  1%|          | 1/150 [00:52<2:10:12, 52.43s/it]

dict_keys(['polygon', 'bbox', 'class_name', 'rotated_polygon', 'rotated_bbox', 'crop', 'text', 'polygon_center'])


  1%|▏         | 2/150 [02:52<3:32:34, 86.18s/it]


In [15]:
!python src/pipeline/evaluate.py \
    --annotation_json_path {DATA_JSON_PATH} \
    --ann_class_names pupil_text pupil_comment teacher_comment \
    --pred_class_names handwritten_text_shrinked_mask1 \
    --pred_jsons_dir {SAVE_FOLDER}

acc_avg.avg=0.02727272727272727 wer_avg.avg=0.981651376146789 cer_avg.avg=0.9968152866242039
acc_avg.avg=0.022292993630573247 wer_avg.avg=0.9871396477577986 cer_avg.avg=0.9746022191434857
acc: 0.0223
wer: 0.9871
cer: 0.9746



100%|██████████| 150/150 [01:45<00:00,  1.42it/s]
